<a href="https://colab.research.google.com/github/chinmaidosapati/TRAVEL_CHAT_BOT/blob/main/travel_assistant_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_core langchain_groq langchain_community langchain langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 975.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 8.6 MB/s eta 0:00:00


In [2]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

**Define Agent**

In [3]:
from typing import TypedDict, Annotated, List, Union

class Message(TypedDict):
    """Represents a generic message, can be extended for specific types."""
    content: str
    sender: str  # e.g., "user" or "ai"

class PlannerState(TypedDict):
    """Stores the state of the trip planner."""
    messages: Annotated[List[Message], "Conversation history"]
    city: str
    interests: List[str]
    itinerary: str





In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
result = llm.invoke("What is Multi AI agent")
result.content


'A Multi-Agent System (MAS) is a system composed of multiple autonomous agents that interact with each other and their environment to achieve common or individual goals. In the context of Artificial Intelligence (AI), a Multi-AI Agent refers to a system that consists of multiple AI agents that work together to solve complex problems, make decisions, or perform tasks.\n\nCharacteristics of Multi-AI Agents:\n\n1. **Autonomy**: Each AI agent has its own autonomy to make decisions and take actions.\n2. **Interoperability**: AI agents can communicate and interact with each other to achieve common goals.\n3. **Distributed Problem-Solving**: Multiple AI agents work together to solve complex problems that are difficult or impossible for a single agent to solve.\n4. **Decentralized Decision-Making**: Decision-making is distributed among multiple AI agents, allowing for more robust and flexible decision-making.\n\nTypes of Multi-AI Agents:\n\n1. **Cooperative Agents**: Agents work together to ac

In [5]:
from langchain.prompts import ChatPromptTemplate

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Create a day trip itinerary for {city} based on user's interests: {interests}. Provide a brief, bulleted itinerary "),
    ("human", "Create a itinerary for my day trip"),

])

Define agent function


In [6]:
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip:")
    user_message = input("Your Input:")
    return {
        **state,
        "city": user_message,
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }

def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter the interests you have for your day trip to {state['city']} (comma-separated):")
    user_message = input("Your Input:")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"Creating an itinerary for {state['city']} based on interests: {', '.join(state['interests'])}")
    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], interests=', '.join(state['interests'])))
    print("Final itinerary:")
    print(response.content)
    # user_message = input("Your Input:")  # Removed as it's not used
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=response.content)],
        "itinerary": response.content
    }

workflow = StateGraph(PlannerState)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)

app = workflow.compile()


In [7]:
def travel_planner(user_request: str):
  print("Initial Request: {user_request}\n")
  state = {
      "messages": [HumanMessage(content=user_request)],
      "city": "",
      "interests": [],
      "itinerary": ""
  }
  for output in app.stream(state):
    pass



In [ ]:
user_request = "I want to plan a day trip"
travel_planner(user_request)


TypeError: travel_planner() missing 1 required positional argument: 'interests'

In [8]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00


In [9]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
import gradio as gr

from typing import TypedDict, Annotated, List, Union

class Message(TypedDict):
    """Represents a generic message, can be extended for specific types."""
    content: str
    sender: str  # e.g., "user" or "ai"

class PlannerState(TypedDict):
    """Stores the state of the trip planner."""
    messages: Annotated[List[Message], "Conversation history"]
    city: str
    interests: List[str]
    itinerary: str

from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)
result = llm.invoke("What is Multi AI agent")
result.content

from langchain.prompts import ChatPromptTemplate # Import ChatPromptTemplate from langchain.prompts

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Create a day trip itinerary for {city} based on user's interests: {interests}. Provide a brief, bulleted itinerary "),
    ("human", "Create a itinerary for my day trip"),

])
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip:")
    user_message = input("Your Input:")
    return {
        **state,
        "city": user_message,
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }

def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter the interests you have for your day trip to {state['city']} (comma-separated):")
    user_message = input("Your Input:")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state["messages"] + [HumanMessage(content=user_message)]
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"Creating an itinerary for {state['city']} based on interests: {', '.join(state['interests'])}")
    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], interests=', '.join(state['interests'])))
    print("Final itinerary:")
    print(response.content)
    # user_message = input("Your Input:")  # Removed as it's not used
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=response.content)],
        "itinerary": response.content
    }

workflow = StateGraph(PlannerState)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)

app = workflow.compile()
def travel_planner(city: str, interests: str):
    state = {
        "messages": [HumanMessage(content=f"Plan a trip to {city} with interests {interests}")],
        "city": city,
        "interests": [i.strip() for i in interests.split(",")],
        "itinerary": ""
    }
    state = create_itinerary(state)
    return state["itinerary"]

interface = gr.Interface(
    fn=travel_planner,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city for your day trip"),
        gr.Textbox(label="Enter your interests (comma-separated)"),
    ],
    outputs=gr.Textbox(label="Generated Itinerary"),
    title="Travel Itinerary Planner",
    description="Enter a city and your interests to generate a personalized day trip itinerary"
)

interface.launch()











/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1114: UserWarning: Cannot load Yntec/HaleyCH_Theme_Orange_Green. Caught Exception: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0ec63b70710b14de7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
